Game https://en.wikipedia.org/wiki/15_puzzle

Objective
- Solve efficiently usign path search

Cost
- the total number of actions eval

In [7]:
import numpy as np
import random
from tqdm.auto import tqdm
import math
from dataclasses import dataclass

In [8]:
# Settings
PROBLEM_SIZE = 3
PUZZLE_SQUARES = pow(PROBLEM_SIZE, 2)
SOLUTION = [(n + 1) % PUZZLE_SQUARES for n in range(PUZZLE_SQUARES)]
PUZZLE = SOLUTION[:]
print(f"SOLUTION: {SOLUTION}")
print(f"PUZZLE: {PUZZLE}")

SOLUTION: [1, 2, 3, 4, 5, 6, 7, 8, 0]
PUZZLE: [1, 2, 3, 4, 5, 6, 7, 8, 0]


In [9]:
def _print_board(board : list):
    """
    Simple board printer.
    Taks a list and print it like a matrix.
    """
    for i in range(PROBLEM_SIZE):
        a = []
        for j in range(PROBLEM_SIZE):
            a.append(board[i*PROBLEM_SIZE+j])
        print(a)
_print_board(SOLUTION)

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]


# Classes

In [10]:
@dataclass
class Action:
    board : np.ndarray # | Current board
    distance : int = 0 # | heuristic distance between the node and the solution
    depth : int = 0    # | tree's depth

# Utility functions

In [11]:
def _get_row(index) -> int:
    """
    Gets the matrix row from the list index.
    """
    return math.floor(index / PROBLEM_SIZE)

def _get_col(index) -> int:
    """
    Gets the matrix col from the list index.
    """
    return index - _get_row(index) * PROBLEM_SIZE

def _get_index(row, col) -> int:
    """
    Get matrix coordinate and return the list index.
    """
    return row * PROBLEM_SIZE + col

def _get_empty_cell(board):
    for n in range(PUZZLE_SQUARES):
        if board[n] == 0:
            return n

def _swap(a, b, board):
    """
    Swap 2 list elements taken their indices.
    """
    tmp = board[a]
    board[a] = board[b]
    board[b] = tmp

def _cost(board):
    """
    Compute the cost of the current array as the distance from the GOAL.
    """
    cost = 0
    for i in range(PUZZLE_SQUARES):
        if board[i] != 0:
            if i < board[i]:
                cost += board[i] - (i+1)
            else:
                cost += (i+1) - board[i]
        # else:
        #     cost += PUZZLE_SQUARES - (i+1)
    return cost

def _evaluate_actions(action):
    """
    Return all the possibile avaible actions for the current states.
    """
    # get coordinate
    index = _get_empty_cell(action.board)
    x, y = _get_row(index), _get_col(index)
    actions = list()

    # move up
    if x > 0:
        up = Action(action.board[:], depth=action.depth + 1) 
        _swap(index, _get_index(x-1, y), up.board)
        up.distance = _cost(up.board)
        actions.append(up)
    # move right
    if y < PROBLEM_SIZE - 1:
        right = Action(action.board[:], depth=action.depth + 1) 
        _swap(index, _get_index(x, y+1), right.board)
        right.distance = _cost(right.board)
        actions.append(right)
    # move down 
    if x < PROBLEM_SIZE - 1:
        down = Action(action.board[:], depth=action.depth + 1) 
        _swap(index, _get_index(x+1, y), down.board)
        down.distance = _cost(down.board)
        actions.append(down)
    # move left
    if y > 0:
        left = Action(action.board[:], depth=action.depth + 1) 
        _swap(index, _get_index(x, y-1), left.board)
        left.distance = _cost(left.board)
        actions.append(left)
    return actions

def _shuffle(action, moves):
    shuffled_action = action
    for i in range(moves):
        new_actions = _evaluate_actions(shuffled_action)
        random.shuffle(new_actions)
        shuffled_action = new_actions.pop()
    return shuffled_action

shuffled = Action(PUZZLE)
shuffled = _shuffle(shuffled, pow(PROBLEM_SIZE, 2)*(PROBLEM_SIZE-1))
shuffled.depth = 0
print(f"INITIAL PUZZLE: {PUZZLE}")
print(f"SHUFFELED PUZZLE: {shuffled.board}")

INITIAL PUZZLE: [1, 2, 3, 4, 5, 6, 7, 8, 0]
SHUFFELED PUZZLE: [1, 2, 0, 7, 4, 3, 5, 8, 6]


In [12]:
def _search(starting_action):
    fronteer = list()
    visited = list()
    current_state = starting_action
    total_cost = 0
    while not np.array_equal(current_state.board, SOLUTION):
        visited.append(current_state)
        for a in _evaluate_actions(current_state):
            in_fronteer = False
            for b in fronteer:
                if np.array_equal(a.board, b.board) and a.depth < b.depth:
                    b.depth = a.depth
                    in_fronteer = True
                    break
            if not in_fronteer and a not in visited:
                fronteer.append(a)
        fronteer.sort(key = lambda x : -(x.distance + x.depth))
        current_state = fronteer.pop()
        total_cost += 1
    return current_state, total_cost

print("\n === INITIAL STATE === \n")
_print_board(shuffled.board)
sol = _search(shuffled)
print("\n === SOLUTION === \n")
print(f"steps: {sol[1]}")
print(f"distance: {sol[0].distance}")
print(f"depth: {sol[0].depth}")
_print_board(sol[0].board)


 === INITIAL STATE === 

[1, 2, 0]
[7, 4, 3]
[5, 8, 6]

 === SOLUTION === 

steps: 11
distance: 0
depth: 8
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]
